In [27]:
import numpy as np

In [28]:
M = 20  # kg
g = 9.8 # m / s^2
m = 0.5 # kg
l = 0.5 # m

In [29]:
v1 = ((M + m) * g) / (M * l)
v2 = -(m * g) / M

c1 = -1 / (M * l)
c2 = 1 / M

In [30]:
A = np.array([[0, 1, 0, 0], [0, 0, v2, 0], [0, 0, 0, 1], [0, 0, v1, 0]])
B = np.array([[0], [c2], [0], [c1]])

In [31]:
print(np.shape(A))
print(np.shape(B))

(4, 4)
(4, 1)


## Controllability

CM = [B AB A<sup>2</sup>B A<sup>3</sup>B]

In [32]:
CM = np.array([[B], [A @ B], [A @ A @ B], [A @ A @ A @ B]]).reshape(4, 4)

In [33]:
print((CM))

[[ 0.      0.05    0.     -0.1   ]
 [ 0.05    0.     -0.1     0.    ]
 [ 0.      0.0245  0.     -2.009 ]
 [ 0.0245  0.     -2.009   0.    ]]


In [34]:
rank = np.linalg.matrix_rank(CM)
print("Rank of CM: ", rank)

Rank of CM:  4


> Rank of CM = 4, therefore controllable

## Stability


In [35]:
print(A)

[[ 0.     1.     0.     0.   ]
 [ 0.     0.    -0.245  0.   ]
 [ 0.     0.     0.     1.   ]
 [ 0.     0.    20.09   0.   ]]


In [36]:
eigenvalues = np.linalg.eigvals(A)

print("Eigenvalues of matrix A:", eigenvalues)

Eigenvalues of matrix A: [ 0.          0.          4.48218697 -4.48218697]


>One eigen value is positive, so the system is unstable

## Policy Iteration


In [37]:
Q = np.array([[100, 0, 0, 0], [0, 100, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
R = np.array([[0.01]])


In [38]:
print("A", A)
print("B", B)

A [[ 0.     1.     0.     0.   ]
 [ 0.     0.    -0.245  0.   ]
 [ 0.     0.     0.     1.   ]
 [ 0.     0.    20.09   0.   ]]
B [[ 0.  ]
 [ 0.05]
 [ 0.  ]
 [-0.1 ]]


In [39]:
F = [[ 0.        ],
 [ 0.01276981],
 [ 0.        ],
 [-0.02003184]]

F = np.array(F).reshape(1, 4)
# F1 = [1, 1, 1, 1]
# F1 = np.array(F1).reshape(1, 4)
# # print eigenvalues of A + BF
# epsilon=0.01
# while(1):
#     if(max(np.linalg.eigvals(A + B @ F)) < 1 and min(np.linalg.eigvals(A + B @ F)) > -1):
#         break
#     F = F- epsilon*F1
#     print("Eigenvalues of A + BF:", np.linalg.eigvals(A + B @ F))
print("Eigenvalues of A + BF:", np.linalg.eigvals(A + B @ F))

Eigenvalues of A + BF: [ 0.00000000e+00  6.22917522e-04  4.48319660e+00 -4.48117784e+00]


In [26]:
def policy_iteration(A, B, Q, R, F):
    # P is I in 4x4
    P = np.identity(4)

    
    for i in range(10000):
        for i in range(100):
            P_new = Q + F.T @ R @ F + (A + B @ F).T @ P @ (A + B @ F)
            if np.allclose(P, P_new):
                break
            P = P_new

        F = -np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A
        

    return F

 
F = policy_iteration(A, B, Q, R, F)

print("F", F)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 4)

In [211]:
import cv2

class InvertedPendulum:
    def __init__(self):
        f = 0

    def step( self, state_vec, t=None ):
        """ state vector :
                x0 : position of the cart
                x1 : veclocity of the cart
                x2 : angle of pendulum. In ref frame with x as forward of the cart and y as up. Angile with respect to ground plane
                x3 : angular velocity of the pendulum
        """
        CART_POS = state_vec[0]
        BOB_ANG  = state_vec[2]*180. / np.pi # degrees
        LENGTH_OF_PENDULUM = 110.

        IM = np.zeros( (512, 512,3), dtype='uint8' )

        # Ground line
        cv2.line(IM, (0, 450), (IM.shape[1], 450), (19,69,139), 4 )


        # Mark ground line
        XSTART = -5.
        XEND = 5.
        for xd in np.linspace( XSTART, XEND, 11 ):
            x = int(   (xd - XSTART) / (XEND - XSTART) * IM.shape[0]   )

            cv2.circle( IM, (x, 450), 5, (0,255,0), -1 )

            cv2.putText(IM, str(xd), (x-15,450+15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1);


        # Draw Wheels of the cart
        wheel_1_pos = int(   (CART_POS - 1.2 - XSTART) / (XEND - XSTART) * IM.shape[0]   )
        wheel_2_pos = int(   (CART_POS + 1.2 - XSTART) / (XEND - XSTART) * IM.shape[0]   )

        cv2.circle( IM, (wheel_1_pos, 415), 25, (255,255,255), 6 )
        cv2.circle( IM, (wheel_2_pos, 415), 25, (255,255,255), 6 )
        cv2.circle( IM, (wheel_1_pos, 415), 2, (255,255,255), -1 )
        cv2.circle( IM, (wheel_2_pos, 415), 2, (255,255,255), -1 )

        # Cart base
        cart_base_start = int(   (CART_POS - 2.5 - XSTART) / (XEND - XSTART) * IM.shape[0]   )
        cart_base_end   = int(   (CART_POS + 2.5 - XSTART) / (XEND - XSTART) * IM.shape[0]   )

        cv2.line( IM, (cart_base_start, 380), (cart_base_end, 380), (255,255,255), 6 )

        # Pendulum hinge
        pendulum_hinge_x = int(   (CART_POS - XSTART) / (XEND - XSTART) * IM.shape[0]   )
        pendulum_hinge_y = 380
        cv2.circle( IM, (pendulum_hinge_x, pendulum_hinge_y), 10, (255,255,255), -1 )


        # Pendulum
        pendulum_bob_x = int( LENGTH_OF_PENDULUM * np.cos( BOB_ANG / 180. * np.pi ) )
        pendulum_bob_y = int( LENGTH_OF_PENDULUM * np.sin( BOB_ANG / 180. * np.pi ) )
        cv2.circle( IM, (pendulum_hinge_x+pendulum_bob_x, pendulum_hinge_y-pendulum_bob_y), 10, (255,255,255), -1 )
        cv2.line( IM, (pendulum_hinge_x, pendulum_hinge_y), (pendulum_hinge_x+pendulum_bob_x, pendulum_hinge_y-pendulum_bob_y), (255,255,255), 3 )

        # Mark the current angle
        angle_display = BOB_ANG % 360
        if( angle_display > 180 ):
            angle_display = -360+angle_display
        cv2.putText(IM, "theta="+str( np.round(angle_display,4) )+" deg", (pendulum_hinge_x-15, pendulum_hinge_y-15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1)


        # Display on top
        if t is not None:
            cv2.putText(IM, "t="+str(np.round(t,4))+"sec", (15, 15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1)
            cv2.putText(IM, "ANG="+str(np.round(BOB_ANG,4))+" degrees", (15, 35), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1)
            cv2.putText(IM, "POS="+str(np.round(CART_POS,4))+" m", (15, 55), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1)



        return IM

In [212]:
sys = InvertedPendulum()

# Initial state
x0 = np.array([0, 0, np.pi/6, 0.])

# U = F @ x, loop and store U and x
x = x0
U = []
X = []

for i in range(100):
    u = -F @ x
    x_dot = A @ x + B @ u
    
    x = x + x_dot * 0.01

    U.append(u)
    X.append(x)

In [213]:
# print eig values of A + BF
print("Eigen values of A + BF", np.linalg.eigvals(A + B @ F))

Eigen values of A + BF [ 0.          0.          0.22580797 -0.22580797]


In [214]:
syst = InvertedPendulum()

for i in range(len(X)):
    rendered = syst.step( [X[i][0], X[i][1], X[i][2], X[i][3]], t=i*0.01)
    print(X[i][2])
    cv2.imshow( 'im', rendered )

    # Press q to exit

    if cv2.waitKey(5) == ord('q'):
        break

0.5235987755982988
0.5256999256883176
0.5299022258683552
0.5362141078458689
0.5446524350357729
0.5552425363960468
0.568018274098951
0.5830221451736297
0.6003054173916582
0.6199282998034171
0.6419601484705879
0.666479708079116
0.6935753902582253
0.7233455895750533
0.755899038321773
0.791355201363257
0.8298447124690104
0.8715098537138551
0.9165050796983243
0.9649975885125531
1.017167941547312
1.0732107344434023
1.133335321666655
1.1977665974009881
1.266745835667167
1.3405315928009176
1.4194006756617035
1.503649179193724
1.5935935972244588
1.6895720106643874
1.7919453575654103
1.9010987898060985
2.0174431215003903
2.1414163745739794
2.273485427320715
2.414147772141257
2.563933389079487
2.723406742210295
2.8931689063970705
3.0738598324301933
3.2661607590810022
3.4707967811610088
3.6885395832656958
3.9202103495082676
4.166682860213593
4.428886787248817
4.707811200417367
5.004508298140188
5.320097376495021
5.65576905158458
6.012789751160987
6.392506492450409
6.7963519642023495
7.225849932136